# Sample commands for MagellanMapper tasks

This notebook demonstrates using MagellanMapper through its command-line interface. You can use the script in various ways:
1. Copy the commands to your own Bash shell script
1. Modify this notebook with your own image paths and run blocks for your desired tasks

## Prereqs

- We assume that you've [installed MagellanMapper](https://github.com/sanderslab/magellanmapper#installation) in a Python environment or from the standalone installer
- If you're using a Python environment, activate it (eg `conda activate mag` or `source <path-to-venv>/bin/activate`) before running this Notebook
- Paths are relative to the `magellanmapper` folder
- Running this Notebook requires the [Bash kernel](https://github.com/takluyver/bash_kernel) for Jupyter Notebook, which can be installed by:

In [ ]:
pip install bash_kernel
python -m bash_kernel.install

## Set up image paths

First, let's set up variables to your image paths.

In [ ]:
# Path to your image file.
img_to_import="/base/path/to/your/image.czi"

# Main image base path. The "base path" is the name of the original image
# file without an extension. MagellanMapper typically outputs files named
# based on it so that you often only need to specify the base path instead of
# exact filenames. For example, after importing img_to_import, the base
# path "/base/path/to/your/image" can be used for most commands.
img="${img_to_import%.*}"

# Downsampled image path. The exact path depends on the type of downsampling
# performed.
img_resized="${img}_resized(456,528,320)"

# Atlas profile for registration tasks. Common profiles are "abaccfv3" for
# the Allen CCFv3 atlas, "abae18pt5" for the Allen Developing Mouse Brain E18.5
# atlas, and "whsrat" for the Waxholm Space rat atlas.
reg_profile=abaccfv3

# set working directory to MagellanMapper folder
if [[ -z "$BASE_DIR" ]]; then
  BASE_DIR="${PWD}/.."
fi
cd "$BASE_DIR"
echo "Set up paths"

## Image import

MagellanMapper typically requires images to be imported into a NumPy format for faster access and lower memory usage. We use BioFormats to import from many formats, including proprietary microscopy formats.

To import from a microscopy file, we run the `import_only` processing task. It assumes that the image format includes metadata, but you can also specify metadata as below. The `-v` option is for verbose output, which is not necessary but can help with troubleshooting.

In [ ]:
./run.py --img "$img_to_import" --proc import_only -v

You can manually specify metadata, which will take precedence over any corresponding settings discovered in the file. `resolutions` are image resolutions in x,y,z order. `magnification` and `zoom` are microscope objective values. See `--set_meta` in the [CLI reference](https://magellanmapper.readthedocs.io/en/latest/cli.html#command-line-argument-reference) for more metadata options.

In [ ]:
./run.py --img "$img_to_import" --proc import_only -v \
  --set_meta resolutions=10.52,10.52,10 magnification=0.63 zoom=1.0

Both single- and multi-plane TIF files can be imported into a volumetric NumPy file. To import a series of TIF files, their filenames should have the format: `name_ch_0.tif`, `name_ch_1.tif`, etc.

In [ ]:
./run.py --img "${img_to_import%.*}.tif" --proc import_only -v

To stitch multi-tile images, use the `pipelines.sh` script instead as described below.

To view the image after import:

In [ ]:
./run.py --img "$img"